In [2]:
import cv2
import numpy as np

# Load the Haar Cascade for face detection
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml');

# Load the pre-trained SSD model for face detection
ssd_net = cv2.dnn.readNetFromCaffe(
    'deploy.prototxt.txt',  # Path to the prototxt file
    'res10_300x300_ssd_iter_140000.caffemodel'  # Path to the model file
)

cap=cv2.VideoCapture(0);

while True:
    # Read a frame from the camera
    ret, frame = cap.read();
    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read frame.")
        break;
    # Perform face detection using Haar Cascades
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # faces_haar = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Perform face detection using SSD
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104, 117, 123))
    ssd_net.setInput(blob)
    detections_ssd = ssd_net.forward();


    # Process SSD detections
    for i in range(detections_ssd.shape[2]):
        confidence = detections_ssd[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            box = detections_ssd[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype(int)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, 'SSD', (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    print(frame);
    # Display the frame with detections
    cv2.imshow('Face Detection', frame);
    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break;
# Release the camera and close all OpenCV windows
cap.release();
cv2.destroyAllWindows();